# Description

```
take IA mode to engage with the enemy.

every 5 seconds:
    send random tactiques
        - set_plane_thrust()
        - set_plane_brake ()
        - set_plane_flaps ()
        - activate_post_combustion ()
        - set_plane_pitch ()
        - set_plane_roll ()
        - set_plane_yaw ()
        - stabilize_plane ()
```

Dogfight Sandbox client example
This script show how to use the network mode to controls aircrafts.
Before starts this script, Dogfight Sandbox must be running in "Network mode"
dogfight_client.py is the library needed to communicate with DogFight sandbox

# Constants

In [ ]:
IP = '172.16.17.116'
PORT = 50888
NR_OF_EPISODES = 1000
MAX_EPISODE_TO_SET_CONFIG = 5
FREQ_SET_CONFIG = 5

# Libraries

In [ ]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('../src/network_client_example'))

import dogfight_client as dgfc

import json

import pandas as pd

import random

import sqlite3

import time

from tqdm import tqdm

# Connect to Simulator

In [ ]:
# Enter the IP and port displayed in top-left corner of DogFight screen
dgfc.connect(IP, PORT)

time.sleep(2)

# Get the whole planes list in DogFight scene
# returns a list that contains planes id
planes = dgfc.get_planes_list()
print(str(planes))

dgfc.disable_log()

# Get the id of the plane you want to control
our_id = planes[0]

# Set All Users to IA Mode

In [ ]:
# active all users
for p_id in planes:
    dgfc.activate_ia(p_id)

# UDFs

In [ ]:
def make_post_combustion(plane_id, val):
    '''
    In order to make combustion, thrust level should 100.
    It ensures this.
    '''

    if val == -1:
        return
        
    # Set plane thrust level (0 to 1)
    dgfc.set_plane_thrust(plane_id, 100)
    
    # Wait until plane thrust = 1\
    t = 0
    while t < 1:
    	plane_state = dgfc.get_plane_state(plane_id)
    	t = plane_state['thrust_level']
    
    # Activate the post-combustion (increases thrust power) 
    dgfc.activate_post_combustion(plane_id)

In [ ]:
def get_random_controls():
    '''
    Generates random control sets.
    '''
    control_set = {
        'set_plane_thrust': random.uniform(0, 1),
        'set_plane_brake': random.uniform(0, 1),
        'make_post_combustion': random.choice([-1, 1]),
        'set_plane_pitch': random.uniform(-1.0, 1.0),
        'set_plane_roll': random.uniform(-1.0, 1.0),
        'set_plane_yaw': random.uniform(-1.0, 1.0)
    }

    seconds = random.randint(0, MAX_EPISODE_TO_SET_CONFIG)

    return control_set, seconds

In [ ]:
def set_random_control(p_id: str):
    random_config, random_seconds  = get_random_controls()

    to_return = random_config.copy()
    to_return['random_seconds'] = random_seconds
    if random_seconds == 0:
        to_return

    dgfc.deactivate_ia(our_id)
    for keys, vals in random_config.items():
        if keys in dir(dgfc):
            str_func = f'dgfc.{keys}("{p_id}", {vals})'
        else:
            str_func = f'{keys}("{p_id}", {vals})'
        
        eval(str_func)

    time.sleep(random_seconds)
    dgfc.stabilize_plane(our_id)
    dgfc.activate_ia(our_id)
    return to_return

# Simulate

In [ ]:
# wait until our aircraft takes off.
p = 0
while p < 50:
	plane_state = dgfc.get_plane_state(our_id)
	p = plane_state['altitude']

In [ ]:
simulation_id = time.time()
all_episodes = []
all_adjustments = []
for i in tqdm(range(NR_OF_EPISODES)):
    log_timestamp = time.time()
    log_id = i

    # reset the health and rearm the aircrafts.
    for plane_id in planes:
        dgfc.set_health(plane_id, 1)
        dgfc.rearm_machine(plane_id)

    # adjust our aircrafts randomly
    if i % FREQ_SET_CONFIG == 0:
        dic_adjusted = set_random_control(p_id=our_id)
        dic_adjusted['simulation_id'] = simulation_id
        dic_adjusted['log_id'] = log_id
        dic_adjusted['log_timestamp'] = log_timestamp
        dic_adjusted['log_machine_id'] = our_id
        all_adjustments.append(dic_adjusted)

    # get statistics of the aircrafts
    single_episode = []
    for plane_id in planes:
        plane_state = dgfc.get_plane_state(our_id)

        plane_state['simulation_id'] = simulation_id
        plane_state['log_id'] = log_id
        plane_state['log_timestamp'] = log_timestamp
        plane_state['log_machine_id'] = plane_id

        single_episode.append(plane_state)

    all_episodes.append(single_episode)

    time.sleep(2)

dgfc.disconnect()

# Save

In [ ]:
# Create DB connection
conn = sqlite3.connect(os.path.join('..', 'data', 'v1_logs.db'))
cursor = conn.cursor()

for arr in all_episodes:
    df = pd\
    .DataFrame(arr)\

    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, list)).any():
            df[col] = df[col].apply(json.dumps)

    df.to_sql('aircraft_states', conn, if_exists='append', index=False)

df_2 = pd.DataFrame(all_adjustments)
df_2.to_sql('aircraft_adjustments', conn, if_exists='append', index=False)

conn.close()